In [1]:
import pickle
import sklearn

In [32]:
file_object = open('pra.pkl', 'rb')
data2 = pickle.load(file_object)
file_object.close()

UnpicklingError: pickle data was truncated

In [22]:
for key, value in data2.items():
    print (key, len(value))

SOYABEENS 127243
CORN 103574
DEVELOPED 7035
FOREST 8087
WATER BODY 4321
ALFALFA 311
OATS 25


In [27]:
data = data2

In [28]:
data = {key:value[:2000] for key, value in data.items() if len(value) >= 2000}

In [29]:
from statistics import mean
from random import shuffle
import numpy as np
for i in range(45):
    lin = [l[i] for value in data.values() for l in value]
    mins = min(lin)
    maxs = max(lin)
    means = mean(lin)
    for key, value in data.items():
        for example in value:
            example[i] = np.float64((example[i] - means)/(maxs-mins))

C:\Users\prabh\Anaconda3\lib\site-packages\ipykernel_launcher.py:11: RuntimeWarning: overflow encountered in ushort_scalars
  # This is added back by InteractiveShellApp.init_path()


In [30]:
training_examples = []
training_labels = []
test_examples = []
test_labels = []
cv_examples = []
cv_labels = []
i = 0;
for key, value in data.items():
    training_examples = training_examples + value[:1800]
    cv_examples = cv_examples + value[1800:1900]
    test_examples = test_examples + value[1900:]
    training_labels = training_labels + [i for v in range(1800)]
    cv_labels = cv_labels + [i for v in range(100)]
    test_labels = test_labels + [i for v in range(100)]
    print(i,':', key)
    i+=1

0 : SOYABEENS
1 : CORN
2 : DEVELOPED
3 : FOREST
4 : WATER BODY


In [31]:
from sklearn.utils import shuffle as shf
training_examples, training_labels  = shf(training_examples, training_labels)

In [ ]:
from sklearn import svm
clf = svm.SVC(C=1.0, 
              kernel=’rbf’, 
              degree=3, 
              gamma=’auto’, 
              coef0=0.0, 
              shrinking=True, 
              probability=False, 
              tol=0.001, 
              cache_size=200, 
              class_weight=None, 
              verbose=False, 
              max_iter=-1, 
              decision_function_shape=’ovr’, 
              random_state=None)


In [ ]:
clf.fit(training_examples, training_labels)

In [ ]:
sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels)

In [ ]:
sum(clf.predict(training_examples) == training_labels)/len(training_labels)

In [ ]:
sum(clf.predict(test_examples) == test_labels)/len(test_labels)

In [ ]:
from sklearn.neural_network import MLPClassifier
cnf = MLPClassifier(hidden_layer_sizes=(100, ),
                    activation=’relu’, 
                    solver=’adam’, 
                    alpha=0.0001, 
                    batch_size=’auto’, 
                    learning_rate=’constant’, 
                    learning_rate_init=0.001, 
                    power_t=0.5, 
                    max_iter=200, 
                    shuffle=True, 
                    random_state=None, 
                    tol=0.0001, 
                    verbose=False, 
                    warm_start=False, 
                    momentum=0.9, 
                    nesterovs_momentum=True, 
                    early_stopping=False, 
                    validation_fraction=0.1, 
                    beta_1=0.9, 
                    beta_2=0.999, 
                    epsilon=1e-08

In [ ]:
cnf.fit(training_examples, training_labels)

In [ ]:
sum(clf.predict(cv_examples) == cv_labels)/len(cv_labels)

In [ ]:
sum(clf.predict(training_examples) == training_labels)/len(training_labels)

In [ ]:
sum(clf.predict(test_examples) == test_labels)/len(test_labels)